In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from sklearn.metrics import mean_squared_error
paleta_colores = sns.color_palette()
plt.style.use("fivethirtyeight")
import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files/Graphviz/bin'
from openpyxl import load_workbook
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from scipy.stats import pearsonr
control = 18
workbook = load_workbook(filename=r'C:\Users\harol\OneDrive\Documents\recoleccion_data.xlsx') 
sheet = workbook.active
def crear_caracteristicas(dataframe):
        dataframe["week"] = dataframe.index.isocalendar().week
        dataframe["week"] = dataframe["week"].astype(np.int64)
        return dataframe
ciudad = ['Ji Parana', 'Manaus', 'Sao Luis', 'Parnaiba', 'Juazeiro do Norte', 'Maranguape', 'Sao Vicente', 'Sertaozinho', 'Santa Cruz do Capibaribe', 
          'Aracaju', 'Eunapolis', 'Belo Horizonte', 'Barra Mansa', 'Rio de Janeiro', 'Sao Goncalo', 'Barretos', 'Barueri', 'Guaruja',
         'Tres Lagoas', 'Rondonopolis']
for city in ciudad:
    for a in range(4):
        sheet[f'A{control + a}'] = city
        sheet[f'B{control + a}'] = "XGBoost"
        sheet[f'C{control + a}'] = "Modelo 3 - AR+GT"
        sheet[f'D{control + a}'] = "Entrenamiento"
        sheet[f'A{control + a + 4}'] = city
        sheet[f'B{control + a + 4}'] = "XGBoost"
        sheet[f'C{control + a + 4}'] = "Modelo 3 - AR+GT"
        sheet[f'D{control + a + 4}'] = "Test"
    if city == 'Ji Parana':
        dataframe = pd.read_excel(r'C:\Users\harol\Downloads\dataverse_files\Dengue_selected_cities.xlsx', sheet_name = city, usecols = ["date", "N_cases", "a dengue", "sintomas da dengue", "sintomas dengue", "sobre a dengue", "sintomas de dengue", "Mosquito da dengue", "Mosquito Dengue", "dengue hemorragica", "mosquito", "mosquitos", "percipitation", "humidity" ])
        dataframe.columns = [ "date", "N_casos", "a dengue", "sintomas da dengue", "sintomas dengue", "sobre a dengue", "sintomas de dengue", "Mosquito da dengue", "Mosquito Dengue", "dengue hemorragica", "mosquito", "mosquitos", "percipitation", "humidity" ]
    else:
        dataframe = pd.read_excel(r'C:\Users\harol\Downloads\dataverse_files\Dengue_selected_cities.xlsx', sheet_name = city, usecols = ["date", "N_cases","dengue", "a dengue", "sintomas da dengue", "sintomas dengue", "sobre a dengue", "sintomas de dengue", "Mosquito da dengue", "Mosquito Dengue", "dengue hemorragica", "mosquito", "mosquitos", "percipitation", "humidity" ])
        dataframe.columns = [ "date", "N_casos", "dengue", "a dengue", "sintomas da dengue", "sintomas dengue", "sobre a dengue", "sintomas de dengue", "Mosquito da dengue", "Mosquito Dengue", "dengue hemorragica", "mosquito", "mosquitos", "percipitation", "humidity" ]
    dataframe = dataframe.set_index("date")
    dataframe = dataframe.loc[ dataframe.index > "2011-01-01"]
    
    dataframe = crear_caracteristicas(dataframe)
    df_train, df_test = train_test_split(dataframe, test_size=14, shuffle=False)

    if city == 'Ji Parana':
         X_train, y_train = df_train[['week',"a dengue", "sintomas da dengue", "sintomas dengue", "sobre a dengue",
                      "sintomas de dengue", "Mosquito da dengue", "Mosquito Dengue", "dengue hemorragica", "mosquito", 
                      "mosquitos", "percipitation", "humidity"]].values, df_train['N_casos'].values.T
     
         X_test, y_test = df_test[['week',"a dengue", "sintomas da dengue", "sintomas dengue", "sobre a dengue",
                      "sintomas de dengue", "Mosquito da dengue", "Mosquito Dengue", "dengue hemorragica", "mosquito", 
                      "mosquitos", "percipitation", "humidity"]].values, df_test['N_casos'].values.T
    else:
         X_train, y_train = df_train[['week',"dengue", "a dengue", "sintomas da dengue", "sintomas dengue", "sobre a dengue",
                      "sintomas de dengue", "Mosquito da dengue", "Mosquito Dengue", "dengue hemorragica", "mosquito", 
                      "mosquitos", "percipitation", "humidity"]].values, df_train['N_casos'].values.T
        
         X_test, y_test = df_test[['week',"dengue", "a dengue", "sintomas da dengue", "sintomas dengue", "sobre a dengue",
                      "sintomas de dengue", "Mosquito da dengue", "Mosquito Dengue", "dengue hemorragica", "mosquito", 
                      "mosquitos", "percipitation", "humidity"]].values, df_test['N_casos'].values.T
    
    model = xgb.XGBRegressor( n_estimators = 10000, 
                                     max_depth = 7, 
                                     learning_rate = 0.005,
                                    early_stopping_rounds = 10,
                                    num_parallel_tree = 4
                                   )
    model.fit(X_train, y_train,
                      eval_set = [ (X_train, y_train),(X_test, y_test) ], 
                      verbose = 1000)
    predictions = model.predict(X_test)
    mse = mean_squared_error(y_test, predictions)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, predictions) if r2_score(y_test, predictions) >= 0 else 0.0
    sheet[f'E{control + 4}'] = "RMSE"
    sheet[f'F{control + 4}'] = rmse
    sheet[f'E{control + 5}'] = "R-RMSE"
    sheet[f'F{control + 5}'] = rmse/(np.max(y_test) - np.min(y_test))
    sheet[f'E{control + 6}'] = "R^2"
    sheet[f'F{control + 6}'] = r2
    sheet[f'E{control + 7}'] = "C. Pearson"
    sheet[f'F{control + 7}'] = pearsonr(y_test, predictions)[0]
    predictions = model.predict(X_train)
    mse = mean_squared_error(y_train, predictions)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_train, predictions) if r2_score(y_train, predictions) >= 0 else 0.0
    sheet[f'E{control}'] = "RMSE"
    sheet[f'F{control}'] = rmse
    sheet[f'E{control + 1}'] = "R-RMSE"
    sheet[f'F{control + 1}'] = rmse/(np.max(y_train) - np.min(y_train))
    sheet[f'E{control + 2}'] = "R^2"
    sheet[f'F{control + 2}'] = r2
    sheet[f'E{control + 3}'] = "C. Pearson"
    sheet[f'F{control + 3}'] = pearsonr(y_train, predictions)[0]
    workbook.save(filename=r'C:\Users\harol\OneDrive\Documents\recoleccion_data.xlsx')
    control += 24
    print(city)

[0]	validation_0-rmse:17.47747	validation_1-rmse:0.50463
[9]	validation_0-rmse:16.87447	validation_1-rmse:0.55781
Ji Parana
[0]	validation_0-rmse:865.23221	validation_1-rmse:137.45731
[574]	validation_0-rmse:114.33515	validation_1-rmse:55.18424
Manaus
[0]	validation_0-rmse:80.51568	validation_1-rmse:27.99384
[69]	validation_0-rmse:60.85028	validation_1-rmse:21.59152
Sao Luis
[0]	validation_0-rmse:13.19618	validation_1-rmse:4.10529
[72]	validation_0-rmse:10.30225	validation_1-rmse:2.68771
Parnaiba
[0]	validation_0-rmse:29.15820	validation_1-rmse:9.72853
[218]	validation_0-rmse:14.36916	validation_1-rmse:5.94824
Juazeiro do Norte
[0]	validation_0-rmse:7.88160	validation_1-rmse:11.32232
[554]	validation_0-rmse:1.08520	validation_1-rmse:8.23102
Maranguape
[0]	validation_0-rmse:79.69060	validation_1-rmse:5.11753
[55]	validation_0-rmse:64.17539	validation_1-rmse:4.02609
Sao Vicente
[0]	validation_0-rmse:57.42197	validation_1-rmse:0.89927
[14]	validation_0-rmse:54.30486	validation_1-rmse:1.15